# Import Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from keras.datasets import boston_housing

In [2]:
(X_train , y_train) , (X_test, y_test) = boston_housing.load_data()

In [3]:
print(f'Training Data : {X_train.shape}')
print(f'Test Data : {X_test.shape}')
print(f'Train Sample : {X_train[0]}')
print(f'Test Sample : {X_test[0]}')

Training Data : (404, 13)
Test Data : (102, 13)
Train Sample : [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
Test Sample : [ 18.0846   0.      18.1      0.       0.679    6.434  100.       1.8347
  24.     666.      20.2     27.25    29.05  ]


# Preparing the data

* 1.Way

In [4]:
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std

X_test -= mean
X_test /= std

# Note that the quantities used for normalizing the test data are computed using the
# training data. You should never use in your workflow any quantity computed on the
# test data, even for something as simple as data normalization.

* 2.Way

In [5]:
# from sklearn.preprocessing import StandardScaler # MinMaxScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# X_test = scaler.transform(X_test)

# Building the Neural Network

In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses

def build_simple_model():
    model = models.Sequential()
    model.add(layers.Dense(128, activation = 'relu' , input_shape = (X_train.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(32, activation = 'relu'))
    
    model.add(layers.Dense(1))
    
    return model

# K-Fold Validation

In [10]:
from sklearn.model_selection import KFold

k = 5
num_epochs = 100
all_scores = []

model = build_simple_model()
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.MeanSquaredError(),
              metrics=['mae'])

# Perform K-Fold cross validation
kf = KFold(n_splits=k)
for i, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f'Processing fold #{i}')

    # Split data into training and validation sets
    partial_train_data, val_data = X_train[train_index], X_train[val_index]
    partial_train_targets, val_targets = y_train[train_index], y_train[val_index]

    # Train the model
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)

    # Evaluate the model on the validation set
    _, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

    # Reset the model's weights
    model.set_weights(build_simple_model().get_weights())

Processing fold #0
Processing fold #1
Processing fold #2
Processing fold #3
Processing fold #4


In [11]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')

all_scores : [1.9217571020126343, 2.3364145755767822, 2.1722605228424072, 2.582444190979004, 2.1032001972198486]
mean all scores : 2.2232153177261353


In [19]:
model = build_simple_model()

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.MeanSquaredError(),
              metrics=['mae'])

model.fit(X_train, y_train, epochs=80, batch_size=8, verbose=1)

test_mse_score, test_mae_score = model.evaluate(X_test, y_test)

Epoch 1/80
51/51 [==============================] - 0s 3ms/step - loss: 330.8762 - mae: 15.2810
Epoch 2/80
51/51 [==============================] - 0s 2ms/step - loss: 48.9057 - mae: 4.9657
Epoch 3/80
51/51 [==============================] - 0s 2ms/step - loss: 22.6340 - mae: 3.2926
Epoch 4/80
51/51 [==============================] - 0s 2ms/step - loss: 17.9044 - mae: 3.0373
Epoch 5/80
51/51 [==============================] - 0s 2ms/step - loss: 15.2735 - mae: 2.8034
Epoch 6/80
51/51 [==============================] - 0s 2ms/step - loss: 13.2503 - mae: 2.5478
Epoch 7/80
51/51 [==============================] - 0s 2ms/step - loss: 12.4915 - mae: 2.4868
Epoch 8/80
51/51 [==============================] - 0s 2ms/step - loss: 11.1325 - mae: 2.4153
Epoch 9/80
51/51 [==============================] - 0s 2ms/step - loss: 10.7269 - mae: 2.3339
Epoch 10/80
51/51 [==============================] - 0s 2ms/step - loss: 9.9893 - mae: 2.3062
Epoch 11/80
51/51 [==============================] - 0s 2m

In [ ]:
# Done.